In [ ]:
import numpy as np 

In [ ]:
from AntidoteRecs.gfc import get_claims, load_data

# # Uncommend the block below to run the code and retrieve the claims from Google Fact Check API
# CREDENTIALS = {'API_KEY': '<your_api_key>','VERSION': '<your_version>'}
# claims = get_claims(CREDENTIALS, keywords=['covid-19 vaccines alter human dna'])

corpus = load_data(file_path='./data/gold-claims-collection.csv')
corpus.head()

In [ ]:
# Initialize the vectorizer
from AntidoteRecs.vectorizers import BertVectorizer

# COVID-Twitter-BERT model
model_name = 'digitalepidemiologylab/covid-twitter-bert-v2'
vectorizer = BertVectorizer(model_name=model_name)

fake_claim = 'COVID-19 vaccines alter human DNA'
fake_claim_embedding = vectorizer.transform(fake_claim)

In [ ]:
# (Optional) Load embeddings from numpy array
embeddings = np.load(f'./data/gold-claims-collection-embeddingstwitbert.npy', allow_pickle=True)
corpus['embedding'] = [claim_embedding for claim_embedding in embeddings]

In [ ]:
# Antidote Recommendation Tool: Vector-based Approach
from AntidoteRecs.antidote import get_antidotes

antidotes = get_antidotes(fake_claim, corpus, vectorizer, topK=10)
antidotes

In [ ]:
from AntidoteRecs.claim_topic import extract_topics

kmeans = extract_topics(corpus, n_topics=100, vectorizer=vectorizer)

In [ ]:
# Antidote Recommendation Tool: Topic Modeling Approach
from AntidoteRecs.antidote import get_antidotes

# Get the topic of the fake claim
fake_claim_topic = kmeans.predict([fake_claim_embedding])[0]

antidotes = get_antidotes(fake_claim, corpus, vectorizer, topK=10, topic=fake_claim_topic)
antidotes

In [ ]:
# Pre-emptive Recommendation Tool 
from AntidoteRecs.graphs import extract_graph, user_to_topic_graph
# Construct the User-to-Item Graph
G = extract_graph('/home/igeor/Antidote-Recommendation-Tool/subreddits/CoronavirusUK/')
# Construct the User-to-Topic Graph
G_U2T = user_to_topic_graph(G, corpus, kmeans, vectorizer)

In [ ]:
import numpy as np
from AntidoteRecs.preemptive import predict_next_topic

# Select the user id
author_id = 10
# Predict the next topic
next_topic = predict_next_topic(G_U2T, author_id, method='SVD', topK=10)

# Select the claims that belong to that topic
antidotes = corpus[corpus['topic'] == next_topic]
antidotes['review']